## Tech Challenge - Fase 3

Este modelo de LLM é capaz de responder perguntas sobre produtos da Amazon, com base em um conjunto de dados de títulos e descrições de produtos da base de dados AmazonTitles-1.3MM.

Este modelo foi preparado utilizando o modelo DeepSeek-R1-Distill-Llama-8B, que é um modelo de linguagem pré-treinado com 8 bilhões de parâmetros. A base de dados utilizada foi tratada para que o modelo pudesse entender as perguntas e responder de forma coerente.

Os dados de treinamento estão no seguinte formato:

```
{
    "Question": "Pergunta do usuário",
    "Complex_CoT": "Resoning do modelo",
    "Response": "Resposta do modelo"
}
```

Após o treinamento, ao ser perguntado sobre um produto, o modelo deve responder de forma coerente e clara, utilizando a lógica e o raciocínio (reasoning) para chegar a uma resposta.


## Bibliotecas utilizadas no projeto

- `unsloth`: Ferramenta para realizar o fine-tuning do LLM — O que foi utilizado:
    - `FastLanguageModel` module para otimizar inference e fine-tuning
    - `get_peft_model` Permite habilitar o LoRA (Low-Rank Adaptation) fine-tuning
- `peft`: Suporte ao LoRA-based fine-tuning para LLMs.
- Modules do Hugging FAce:
    - `transformers` Para utilizar nos dados do fine-tuning data e outras tarefas do modelo
    - `trl` Habilita o Supervised Fine-Tuning
    - `datasets` Essa biblioca permite baixar datasets que estão disponíveis no  Hugging Face Hub, neste caso, foi utilizado um dataset criado especificamente para esse modelo: https://huggingface.co/datasets/rickwalking/amazon-titles-reasoning
- `torch`: Deep learning framework utilizado no treino
- `wandb`: Realizar tracking do fine-tuning

## Instalando as as dependências do projeto

In [ ]:
%%capture

!pip install unsloth # installa o unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git # Instalar a versão mais recente do Unsloth!

## Realizando o import dos modules necessários para o fine-tuning

In [ ]:
# Modules para o fine-tuning
from unsloth import FastLanguageModel
import torch # Import PyTorch
from trl import SFTTrainer # Trainer for supervised fine-tuning (SFT)
from unsloth import is_bfloat16_supported # Checks if the hardware supports bfloat16 precision
# Hugging Face modules
from huggingface_hub import login # Lets you login to API
from transformers import TrainingArguments # Defines training hyperparameters
from datasets import load_dataset # Lets you load fine-tuning datasets
# Import weights and biases
import wandb
# # Import kaggle secrets
# from kaggle_secrets import UserSecretsClient

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Verificando a disponibilidade de CUDA
Após instalar as dependências, é necessário verificar a disponibilidade de CUDA.


In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cu124
True


## Carregar o DeepSeek R1 e o Tokenizer

Nesta etapa, será realizado o download do DeepSeek R1 e o tokenizer, usando o `FastLanguageModel.from_pretrained()`.

**Key parameters explained**
```py
max_seq_length = 2048  # Número máximo de tokens por input
dtype = None  # Data type default (auto detect)
load_in_4bit = True  # Habilita o  4-bit quantization – uma otimização de memória
```

**Sobre o  4-bit quantization**

É como comprimir uma imagem de resolução alta para um tamanho menor. O resultado será uma imagem menor, ocupando menos espaço, mas com a mesma qualidade anterior. Essa abordagem realiza um diminuição no tamanho do modelo, sem perder a qualidade e precisão do LLM.


In [ ]:
# Parâmetros para o modelo
max_seq_length = 2048
dtype = None
load_in_4bit = True

# Carregar o DeepSeek R1 e o tokenizer usando o unsloth
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2025.3.17: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

## Testando o  DeepSeek R1 com uma pergunta de um cliente antes do fine-tuning


### System prompt
O system promp é definido e incluí seus placeholders para a pergunta e a resposta do modelo. Este prompt irá guiar o modelo a "pensar" passo a passo e providenciar uma resposta correta.

In [ ]:
# Define a system prompt under prompt_style
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a Amazon products expert with advanced knowledge in books, cellphones, eletronics, media, and more.
Please answer the customer question.

### Question:
{}

### Response:
<think>{}"""

### Rodando o modelo

Neste passo, iremos testar o DeepSeek R1 Model com uma pergunta de um cliente da Amazon, e printar a resposta.
Este processo tem os seguintes passos:

1. **Definir a pergunta**
2. **Formatar a pergunta seguindo o modelo de prompt (`prompt_style`)**
3. **Realizar a tokenização e enviar para a  GPU (`cuda`)**
4. **Gerar resposta usando o modelo LLM**,
5. **Realizar o decode do output**

In [ ]:
# Enviando uma pergunta relacionada a um produto
question = """Is 'Worship with Don Moen' available on VHS??"""

FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!

# Formatar a pergunta usando o structured prompt (`prompt_style`) e depois tokenizar
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Gerar a resposta
outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

# REalizar decode dos tokens de output para linguagem natural
response = tokenizer.batch_decode(outputs)

# Printar a resposta do modelo (after "### Response:")
print(response[0].split("### Response:")[1])


<think>
Okay, so I need to figure out if "Worship with Don Moen" is available on VHS. Let me start by understanding the question. The user is asking about a specific product, which is a video tape, likely from the 90s or early 2000s since VHS was popular then. 

First, I should consider where to look for this information. As an expert, I have access to Amazon's product database. I can check if the title exists and what formats are available. I'll search for "Worship with Don Moen" on Amazon.

Upon searching, I find that "Worship with Don Moen" is available on DVD. I don't see any listings for VHS. However, I should check if there are any used or rare VHS listings. Sometimes products are listed by sellers who have old stock.

If I don't find any VHS listings, I can inform the customer that it's not available on VHS but is available on DVD. I should also consider if the customer might be looking for it specifically on VHS, maybe for compatibility reasons. In that case, I should advise t

## Passo a Passo do fine-tuning

### Passo 1 — Atualizar o system prompt
Uma pequena modificação no estilo de prompt para processar o dataset, adicionando um terceiro placeholder relacionado ao reasoning. `</think>`

In [ ]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a Amazon products expert with advanced knowledge in books, cellphones, eletronics, media, and more.
Please answer the customer question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""


### Passo 2 — Carregamento do dataset

Este Dataset é carregado para realizar o fine-tuning do modelo.
O dataset foi criado a partir de um conjunto de dados de títulos e descrições de produtos da base de dados AmazonTitles-1.3MM, adaptado para o formato de treinamento deste modelo de LLM, o DeepSeek-R1-Distill-Llama-8B.

A geração das perguntas, reasoning e respostas foi gerado com a utilização de script e um prompt específico para o DeepSeek, assim gerando um dataset com 2000 exemplos.
Este detaset está hospoedado no Hugging Face: https://huggingface.co/datasets/rickwalking/amazon-titles-reasoning

Abaixo é possível visualizar as primeiras linhas do dataset:

In [ ]:
dataset = load_dataset("rickwalking/amazon-titles-reasoning", "default", split="train[0:2000]", trust_remote_code=True)
dataset

Repo card metadata block was not found. Setting CardData to empty.


amazon_titles_reasoning.json:   0%|          | 0.00/785k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1107 [00:00<?, ? examples/s]

Dataset({
    features: ['Question', 'Complex_CoT', 'Response'],
    num_rows: 1107
})

In [ ]:
# Mostrar um objeto do dataset
dataset[1]

{'Question': "What should I type into Amazon to find 'Rightly Dividing the Word' paperback edition?",
 'Complex_CoT': "To locate a specific book on Amazon, users typically enter the exact title. Since 'Rightly Dividing the Word' is a distinct title and refers to a paperback edition, typing this title into the search bar will effectively find the desired product. This method works because Amazon's search function prioritizes exact matches when searching by title.",
 'Response': "Type 'Rightly Dividing the Word' into the Amazon search bar."}

In [ ]:
EOS_TOKEN = tokenizer.eos_token  # Definindo o EOS_TOKEN para ser adicionado ao final do texto formatado
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [ ]:
# Definindo função para formatação do prompt
def formatting_prompts_func(batch):  # Recebe batches do dataset
    inputs = batch["Question"]       # Extrai as perguntas
    cots = batch["Complex_CoT"]      # Extrai o chain-of-thought reasoning
    outputs = batch["Response"]      # Extrai a resposta do modelo

    texts = []

    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)

    return {
        "text": texts,  Retorna o texto formatado, pronto para o fine-tuning
    }

In [ ]:
# Update dataset formatting
dataset_finetune = dataset.map(formatting_prompts_func, batched = True)
dataset_finetune["text"][0]

Map:   0%|          | 0/1107 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a Amazon products expert with advanced knowledge in books, cellphones, eletronics, media, and more.\nPlease answer the customer question.\n\n### Question:\nWhat type of story is Autumn Story Brambly Hedge?\n\n### Response:\n<think>\nThe product title 'Autumn Story Brambly Hedge' suggests it is likely a storybook, given the mention of 'Brambly Hedge,' which refers to a children's book series featuring hedgehogs. The context provided mentions it being a research-rich fantasy aimed at small children. Combining these elements, the reasoning leads to the conclusion that it is a fantasy storybook for children.\n</think>\nAutumn Story Brambly Hedge is a research-rich f

### Passo 3 — Configurando o modelo usando LoRA

**Uma explicação intuitiva do LoRA**
Os modelos de linguagem grandes (LLMs) têm milhões ou até bilhões de pesos que determinam como eles processam e geram texto. Ao fazer o fine-tuning de um modelo, geralmente atualizamos todos esses pesos, o que requer enormes recursos computacionais e memória.

O LoRA (Low-Rank Adaptation - Adaptação de Baixo Posto) permite fazer fine-tuning de forma eficiente:

- Em vez de modificar todos os pesos, o LoRA adiciona pequenos adaptadores treináveis em camadas específicas.
- Esses adaptadores capturam conhecimento específico da tarefa enquanto mantêm o modelo original inalterado.
- Isso reduz o número de parâmetros treináveis em mais de 90%, tornando o fine-tuning mais rápido e eficiente em termos de memória.
- Pense em um LLM como uma fábrica complexa. Em vez de reconstruir toda a fábrica para produzir um novo produto, o LoRA adiciona pequenas ferramentas especializadas às máquinas existentes. Isso permite que a fábrica se adapte rapidamente sem interromper sua estrutura central.
- Abaixo, usaremos a função `get_peft_model()`, que significa Parameter-Efficient Fine-Tuning (Fine-Tuning Eficiente em Parâmetros) — esta função envolve o modelo base (model) com modificações LoRA, garantindo que apenas parâmetros específicos sejam treinados.

In [ ]:
# Applicando o LoRA
deep_seek_model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Rank do LoRA: Determina o tamanho dos adaptadores treináveis (maior = mais parâmetros, menor = mais eficiência)
    target_modules=[  # Lista de camadas do transformer onde os adaptadores LoRA serão aplicados
        "q_proj",   # Projeção de Query no mecanismo de self-attention
        "k_proj",   # Projeção de Key no mecanismo de self-attention
        "v_proj",   # Projeção de Value no mecanismo de self-attention
        "o_proj",   # Projeção de saída da camada de atenção
        "gate_proj",  # Usado nas camadas feed-forward (MLP)
        "up_proj",    # Parte da rede feed-forward do transformer (FFN)
        "down_proj",  # Outra parte da rede feed-forward do transformer (FFN)
    ],
    lora_alpha=16,  # Fator de escala para atualizações LoRA (valores mais altos permitem maior influência das camadas LoRA)
    lora_dropout=0,  # Taxa de dropout para camadas LoRA (0 significa sem dropout, retenção total de informação)
    bias="none",  # Especifica se as camadas LoRA devem aprender termos de viés (definir como "none" economiza memória)
    use_gradient_checkpointing="unsloth",  # Economiza memória recalculando ativações em vez de armazená-las (recomendado para fine-tuning com contexto longo)
    random_state=3407,  # Define uma semente para reprodutibilidade, garantindo o mesmo comportamento de fine-tuning em diferentes execuções
    use_rslora=False,  # Se deve usar LoRA com Rank Estabilizado (desativado aqui, significando que LoRA de rank fixo é usado)
    loftq_config=None,  # Quantização de Fine-Tuning de Baixo Bit (LoFTQ) está desativada nesta configuração
)

Unsloth 2025.3.17 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Inicializando `SFTTrainer`, para um supervised fine-tuning trainer.

In [ ]:
# Inicializa o treinador de fine-tuning — Importado usando from trl import SFTTrainer
trainer = SFTTrainer(
    model=deep_seek_model,  # O modelo a ser fine-tunado
    tokenizer=tokenizer,  # Tokenizador para processar entradas de texto
    train_dataset=dataset_finetune,  # Dataset usado para treinamento
    dataset_text_field="text",  # Especifica qual campo no dataset contém o texto de treinamento
    max_seq_length=max_seq_length,  # Define o comprimento máximo de sequência para entradas
    dataset_num_proc=2,  # Usa 2 threads da CPU para acelerar o pré-processamento de dados

    # Define argumentos de treinamento
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Número de exemplos processados por dispositivo (GPU) por vez
        gradient_accumulation_steps=4,  # Acumula gradientes por 4 passos antes de atualizar os pesos
        num_train_epochs=1, # Execução completa de fine-tuning
        warmup_steps=5,  # Aumenta gradualmente a taxa de aprendizado nos primeiros 5 passos
        max_steps=60,  # Limita o treinamento a 60 passos (útil para depuração; aumente para fine-tuning completo)
        learning_rate=2e-4,  # Taxa de aprendizado para atualizações de peso (ajustada para fine-tuning LoRA)
        fp16=not is_bfloat16_supported(),  # Usa FP16 (se BF16 não for suportado) para acelerar o treinamento
        bf16=is_bfloat16_supported(),  # Usa BF16 se suportado (melhor estabilidade numérica em GPUs mais novas)
        logging_steps=10,  # Registra o progresso do treinamento a cada 10 passos
        optim="adamw_8bit",  # Usa otimizador AdamW eficiente em memória no modo 8-bit
        weight_decay=0.01,  # Regularização para prevenir overfitting
        lr_scheduler_type="linear",  # Usa um cronograma linear de taxa de aprendizado
        seed=3407,  # Define uma semente fixa para reprodutibilidade
        output_dir="outputs",  # Diretório onde os checkpoints do modelo fine-tunado serão salvos
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/1107 [00:00<?, ? examples/s]

## Passo 4 — Treinando o modelo!

Aqui será iniciado o fine-tuning do modelo.

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,107 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rickwalking1272 (rickwalking1272-n-a) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,2.160500
20,0.971900
30,0.932700
40,0.854600
50,0.868200
60,0.876000


In [ ]:
# Finalizar as análises do wandb
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▃▁▁▃▁
train/learning_rate,█▇▅▄▂▁
train/loss,█▂▁▁▁▁
total_flos,5248816306225152.0
train/epoch,0.43321
train/global_step,60
train/grad_norm,0.47068
train/learning_rate,0
train/loss,0.876


## Passo 5 — Testar o modelo após o fine-tuning

As respostas do modelo devem estar mais específicas ao dataset que foi utilizado no treinamento

In [ ]:
question = """Is 'Worship with Don Moen' available on VHS?"""

FastLanguageModel.for_inference(deep_seek_model)

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = deep_seek_model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

# Resposta
print(response[0].split("### Response:")[1])


<think>
The product title is 'Worship with Don Moen,' and the context mentions that it is available on both VHS and CD formats. Therefore, the question asks if this specific product is available on VHS.
</think>
Yes, 'Worship with Don Moen' is available on VHS.<｜end▁of▁sentence｜>


In [ ]:
question = """What type of story is Autumn Story Brambly Hedge?"""

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = deep_seek_model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

# Resposta
print(response[0].split("### Response:")[1])


<think>
The product title is 'Autumn Story Brambly Hedge,' which suggests it is a book. The context mentions that this is a sequel to 'Brambly Hedge,' which indicates that the book is part of a series. Therefore, someone looking for a children's storybook in a series might ask about the type of story.
</think>
Autumn Story Brambly Hedge is a children's storybook that continues the adventures of Brambly Hedge, indicating it is part of a series.<｜end▁of▁sentence｜>


In [ ]:
question = """Is 'Mermaids: Nymphs of the Sea' available as a physical book with special features like vellum pages and metallic ink?"""

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = deep_seek_model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)

# Resposta
print(response[0].split("### Response:")[1])


<think>
The product title 'Mermaids: Nymphs of the Sea' suggests it is a book. The context mentions that the book is printed on vellum (a high-quality paper) and uses metallic ink, which are features that enhance the visual appeal of the publication. Therefore, someone looking for this book would want to know if these special features are present.
</think>
Yes, 'Mermaids: Nymphs of the Sea' is available as a physical book with special features, including vellum pages and metallic ink.<｜end▁of▁sentence｜>


## Passo 6 — Salvar o modelo no Hugging Face

O modelo será enviado para o Hugging Face

In [ ]:
new_model_online = "rickwalking/DeepSeek-R1-Products"
deep_seek_model.push_to_hub(new_model_online)
tokenizer.push_to_hub(new_model_online)
deep_seek_model.push_to_hub_merged(new_model_online, tokenizer, save_method="merged_16bit")

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/rickwalking/DeepSeek-R1-Products


README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/rickwalking/DeepSeek-R1-Products
